In [2]:
from selenium import webdriver
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [14]:
chrome_driver_loc = 'C:/Users/nikhi/Downloads/chromedriver'
driver = webdriver.Chrome(chrome_driver_loc)

In [4]:
links = pd.read_csv('C:/Users/nikhi/Dropbox/Jharkhand Minerals/Lessee Profile Links.csv')

In [5]:
links[links['ML/RML']=='AUCTION']

,Dmo Name,Lessee Name,Mines Name,ML/RML,Lessee Code,Mineral,detail_link,status
36,GARHWA,BELCHAMPA SAND MINIG PROP AMIT GUPTA,BELCHAMPA SAND MINIG PROJECT,AUCTION,412141101,SAND,https://portal.jharkhandminerals.gov.in/portal...,Working
84,LOHARDAGA,JANKALYAN VIKASH SAMITI,BHAKSO SAND GHAT,AUCTION,516122601,SAND,https://portal.jharkhandminerals.gov.in/portal...,Working
332,PALAMU,MS HINDALCO INDUSTRIES LTD,MS HINDALCO INDUSTRIES LTD,AUCTION,411310001,COAL,https://portal.jharkhandminerals.gov.in/portal...,Working
369,PALAMU,MS JULANGA BALU GHAT PRO ANUPAM KUMAR SINGH,MS JULANGA BALU GHAT PRO ANUPAM KUMAR SINGH,AUCTION,411153501,SAND,https://portal.jharkhandminerals.gov.in/portal...,Working
583,SARAIKELA KHARSAWAN,NIKHIL AGARWAL,NIKHIL AGARWAL,AUCTION,103516301,SAND,https://portal.jharkhandminerals.gov.in/portal...,Working
...,...,...,...,...,...,...,...,...
3750,RANCHI,SURJEET KUMAR SINGH,SURJEET KUMAR SINGH,AUCTION,514517401,SAND,https://portal.jharkhandminerals.gov.in/portal...,Non-Working
3774,HAZARIBAGH,TOKISUD NORTH COAL MINE,TOKISUD NORTH COAL MINE,AUCTION,307178501,COAL,https://portal.jharkhandminerals.gov.in/portal...,Non-Working
3793,KHUNTI,UNITH KUMAR KASHYAP,TORPA BARKULI,AUCTION,515539101,SAND,https://portal.jharkhandminerals.gov.in/portal...,Non-Working
3794,KHUNTI,UNITH KUMAR KASHYAP,FUDI,AUCTION,515539102,SAND,https://portal.jharkhandminerals.gov.in/portal...,Non-Working


In [11]:
#RML - rptAccordian_ctl00_tblView
#AUCTION - rptAccordian_ctl00_tblView

In [24]:
i = 36
link = links['detail_link'][i]
driver.get(link)

In [6]:
def remove_columns(df):
    cols = list(df.columns)
    for i in range(len(cols)):
        c = cols[i]
        try :
            temp = int(c[c.rfind('.')+1:])
            cols[i] = c[:c.rfind('.')]
        except:
            continue
    return pd.DataFrame([cols]+df.values.tolist())

In [25]:
#LESSEE DETAILS
def get_lessee_details(link):
    linkId = 'LesseeProfile'
    tableId = 'grdLProfile'
    r = requests.get(link.replace('LesseeProfile',linkId))
    soup = BeautifulSoup(r.content)
    pre = ''
    details = {}
    try:
        df = pd.read_html(str(soup.find("table",{"id":tableId})))[1]
    except:
        return details
    for l in df.values:
        if len(set([x for x in l if x==x])) == 1:
            pre = l[0] +' '
            continue
        i=1
        last_detail = 'abcdef'
        while i<len(l):
            if l[i-1]==last_detail:
                i+=1
                continue
            details.update({pre+str(l[i-1]):l[i]})
            last_detail = l[i]
            if l[i] == 'Contact Person Name':
                break
            i+=2
    return details

In [40]:
#LEASE INFO
def get_lease_info(link,ml=1):
    linkId = 'LesseeInfo'
    tableId = 'grdLeaseInfo'
    pre = ''
    details = {}
    r = requests.get(link.replace('LesseeProfile',linkId))
    soup = BeautifulSoup(r.content)
    try:
        df = pd.read_html(str(soup.find("div",{"class":"viewTable"}).find({"table"})))[1]
    except:
        return details
    for l in df.values:
        if len(set([x for x in l if x==x])) == 1:
            pre = l[0] +' '
            continue
        i=1
        if l[0] == 'Non Forest Area (HA)':
            pre = pre+l[0]+' '
            i=2
        last_detail = 'abcdef'
        while i<len(l):
            if l[i-1]==last_detail:
                i+=1
                continue
            details.update({pre+str(l[i-1]):l[i]})
            last_detail = l[i]
            i+=2
    return details

In [27]:
#FOREST CLEARANCE
def get_forest_clearance(link,ml=1):
    linkId = 'ForestClearance'
    tableId = 'grdForClear'
    pre = ''
    details = {}
    r = requests.get(link.replace('LesseeProfile',linkId))
    soup = BeautifulSoup(r.content)
    try:
        df = pd.read_html(str(soup.find("table",{"id":tableId})))[ml]
    except:
        return details
    df = df['Forest Clearance']
    df = remove_columns(df)
    for l in df.values:
        if len(set([x for x in l if x==x])) == 1:
            pre = l[0] +' '
            continue
        i=1
        if l[0] == 'Stage 1 Clearance':
            pre = pre+l[0]+' '
            i=3
        elif l[0] == 'Stage 2 Clearance':
            pre = 'Non-Forest Land Details '+l[0]+' '
            i=3        
        last_detail = 'abcdef'
        while i<len(l):
            if l[i-1]==last_detail:
                i+=1
                continue
            details.update({pre+str(l[i-1]):l[i]})
            last_detail = l[i]
            i+=2
    return details

In [1]:
#ENVIRONMENT CLEARANCE
def get_env_clearance(link,ml=1):
    linkId = 'EnvClearance'
    tableId = 'grdEnv'
    if not ml:
        tableId = 'rptAccordian_ctl00_tblView'
    r = requests.get(link.replace('LesseeProfile',linkId))
    soup = BeautifulSoup(r.content)
    details = {}
    try:
        df = pd.read_html(str(soup.find("table",{"id":tableId})))[0]
    except:
        return details
    for i in df.index:
        for c in df.columns:
            details.update({c+' '+str(i+1):df[c][i]})
    return details

In [29]:
#POLLUTION CLEARANCE
def get_pol_clearance(link):
    linkId = 'PCBClearance'
    tableId = 'grdOspcbML'
    pre = ''
    details = {}
    r = requests.get(link.replace('LesseeProfile',linkId))
    soup = BeautifulSoup(r.content)
    try:
        dfs = pd.read_html(str(soup.find("table",{"id":tableId})))
    except:
        return details
    df = remove_columns(df)
    for j in range(1,len(dfs)):
        df = remove_columns(dfs[j])
        for l in df.values:
            if len(set([x for x in l if x==x])) == 1:
                pre = l[0] +' ' + str(j)+' '
                continue
            i=1
            last_detail = 'abcdef'
            while i<len(l):
                if l[i-1]==last_detail:
                    i+=1
                    continue
                details.update({pre+str(l[i-1]):l[i]})
                last_detail = l[i]
                i+=2
    return details

In [30]:
#SURVEY
def get_survey_details(link):
    linkId = 'Survey'
    tableId = 'grdSurveyML'
    pre = ''
    details = {}
    r = requests.get(link.replace('LesseeProfile',linkId))
    soup = BeautifulSoup(r.content)
    try:
        df = pd.read_html(str(soup.find("table",{"id":tableId})))[1]
    except:
        return details
    df = remove_columns(df)
    for l in df.values:
        if len(set([x for x in l if x==x])) == 1:
            pre = l[0] +' '
            continue
        i=1
        last_detail = 'abcdef'
        while i<len(l):
            if l[i-1]==last_detail:
                i+=1
                continue
            details.update({pre+str(l[i-1]):l[i]})
            last_detail = l[i]
            i+=2
    return details

In [31]:
#MINING PLAN
def get_mining_plan_details(link):
    linkId = 'MiningPlan'
    tableId = 'tbML'
    pre = 'Mining Plan '
    details = {}
    r = requests.get(link.replace('LesseeProfile',linkId))
    soup = BeautifulSoup(r.content)
    try:
        df = pd.read_html(str(soup.find("table",{"id":tableId})))[0].iloc[1:5,1:]
    except:
        return details
    for l in df.values:
        if len(set([x for x in l if x==x])) == 1:
            pre = l[0] +' '
            continue
        i=1
        last_detail = 'abcdef'
        while i<len(l):
            if l[i-1]==last_detail:
                i+=1
                continue
            if pre in l[i-1]:
                details.update({l[i-1]:l[i]})
            else:
                details.update({pre+str(l[i-1]):l[i]})
            last_detail = l[i]
            i+=2
    return details

In [32]:
#MINING PLAN 2
def get_mining_plan_grid_details(link):
    linkId = 'MiningPlan'
    tableId = 'grdMiningPlan'
    r = requests.get(link.replace('LesseeProfile',linkId))
    soup = BeautifulSoup(r.content)
    details = {}
    try:
        df = pd.read_html(str(soup.find("table",{"id":tableId})))[0]
    except:
        return details
    for i in df.index:
        for c in df.columns:
            details.update({c+' '+str(i+1):df[c][i]})
    return details

In [33]:
#GRANT/EXECUTION
def get_grant_details(link):
    linkId = 'Grant'
    tableId = 'grLGrant'
    pre = ''
    details = {}
    r = requests.get(link.replace('LesseeProfile',linkId))
    soup = BeautifulSoup(r.content)
    try:
        df = pd.read_html(str(soup.find("table",{"id":tableId})))[1]
    except:
        return details
    df = remove_columns(df)
    for l in df.values:
        if len(set([x for x in l if x==x])) == 1:
            pre = l[0] +' '
            continue
        i=1
        last_detail = 'abcdef'
        while i<len(l):
            if l[i-1]==last_detail:
                i+=1
                continue
            details.update({pre+str(l[i-1]):l[i]})
            last_detail = l[i]
            i+=2
    return details

In [34]:
#SURFACE RIGHT GRANT/POSSESSION
def get_surface_right_details(link):
    linkId = 'SurfaceRightCL'
    tableId = 'grdSurface'
    pre = ''
    details = {}
    r = requests.get(link.replace('LesseeProfile',linkId))
    soup = BeautifulSoup(r.content)
    try:
        dfs = pd.read_html(str(soup.find("table",{"id":tableId})))
    except:
        return details
    for j in range(1,3):
        df = remove_columns(dfs[j])
        for l in df.values:
            if len(set([x for x in l if x==x])) == 1:
                pre = l[0] +' '
                continue
            i=1
            last_detail = 'abcdef'
            while i<len(l):
                if l[i-1]==last_detail:
                    i+=1
                    continue
                details.update({pre+str(l[i-1]):l[i]})
                last_detail = l[i]
                if '(HA)' in l[i-1]:
                    break
                i+=2
    return details

In [35]:
#DCC
def get_dcc_details(link):
    linkId = 'DCC'
    tableId = 'grdDCC'
    details = {}
    r = requests.get(link.replace('LesseeProfile',linkId))
    soup = BeautifulSoup(r.content)
    try:
        df = pd.read_html(str(soup.find("table",{"id":tableId})))[0]
    except:
        return details
    for i in df.index:
        for c in df.columns:
            details.update({c+' '+str(i+1):df[c][i]})

In [ ]:
#TRIALS

In [38]:
ml = 0
linkId = 'EnvClearance'
tableId = 'grdEnv'
if not ml:
    tableId = 'rptAccordian_ctl00_tblView'
r = requests.get(link.replace('LesseeProfile',linkId))
soup = BeautifulSoup(r.content)
details = {}

In [12]:
tableId

'rptAccordian_ctl00_tblView'

In [39]:
pd.read_html(str(soup.find('div',{'class':'viewTable'}).find('table')))[0]

,Application Date,Letter No.,Order Date,Valid Upto,Mineral Name,Qt. Approved (in MT),Date of Hearing,View File,Remark
0,02 Sep 2017,EC/DEIAA/2016-17/26,07 Jan 2018,23 Oct 2022,SAND,102197.0,23 Oct 2017,NaN,as per general condition


In [18]:
s = soup.find('div',{"class":"viewTable"})

In [27]:
soup.find('div',{'id':'viewTable'})

In [37]:
i = 36
link = links['detail_link'][i]
driver.get(link)

In [44]:
linkId = 'LesseeInfo'
tableId = 'grdLeaseInfo'
pre = ''
details = {}
r = requests.get(link.replace('LesseeProfile',linkId))
soup = BeautifulSoup(r.content)

In [47]:
soup.find("div",{"class":"viewTable"})

<div class="viewTable">
<div>
</div>
</div>

In [46]:
pd.read_html(str(soup.find("div",{"class":"viewTable"}).find({"table"})))[0]

ValueError: No tables found

In [ ]:


for l in df.values:
    if len(set([x for x in l if x==x])) == 1:
        pre = l[0] +' '
        continue
    i=1
    if l[0] == 'Non Forest Area (HA)':
        pre = pre+l[0]+' '
        i=2
    last_detail = 'abcdef'
    while i<len(l):
        if l[i-1]==last_detail:
            i+=1
            continue
        details.update({pre+str(l[i-1]):l[i]})
        last_detail = l[i]
        i+=2
return details